In [1]:
%matplotlib inline

In [15]:
import numpy as np
from scipy.special import zeta
from matplotlib import pyplot as plt

In [140]:
class PowerExpansion(list):
    def __init__(self, *args, **kwds):
        """
        Establishes the API for expansion objects. Since these can,
        and often will, be constructed incrementally, it is preferrable
        to use a stateful object.
        
        A power series expansion is defined as
        
        .. math::
        
            f(x) = \sum_{k=0}^\infty a_n x^n
        
        therefore, only the coefficients need be known to calculate
        the function approximation. This can be done by calling the
        power expansion as a function.
        
        Parameters
        ----------
        This takes the same arguments as list.
        
        Examples
        --------
        
        .. code:: python

            empty = PowerExpansion() # creates an empty expansion
            order_7 = PowerExpansion(7*[0]) # creates a 7th order expansion
        """
        super().__init__(self, *args, **kwds)
        self._generating_function = None
        
    def __call__(self, x):
        x = np.asarray(x)
        index = 0
        # find how many coefficients have been set
        while True:
            try:
                _ = self[index]
                index += 1
            except IndexError:
                break
        # calculate sum_{k=0}^{index} c_k x^k
        ck = np.array(self[:index]) # c_k
        k = np.arange(index)        # k
        # to reduce numerical instabilities at large powers,
        # convert the coefficients to a power of x, i.e.
        # x: x = sign(x)*|x|
        # c_k: c_k = sign(c_k)*x^{log_x(|c_k|)}
        # c_k x^k = sign(c_k)*sign(x)^k*|x|^{k + log_|x|(|c_k|)}
        sign_ck = np.sign(ck)
        sign_x = np.sign(x)
        # |x|
#         X = np.repeat(np.abs(x)[:, np.newaxis], index, axis=-1)
        # |x|^{k + log_|x|(|c_k|)}
        # e^{k ln(|x|) + ln(|c_k|)}
#         xpow = k + np.divide(np.log(np.abs(ck)),
#                              np.log(np.abs(x))[:, np.newaxis])
        xpow = k*np.log(np.abs(x))[:, np.newaxis] + \
                np.log(np.abs(ck))
        X[np.isnan(xpow)] = 0
        X = np.exp**xpow
        # sign(x)^k*...
        X = sign_x[:, np.newaxis]**k * X # broadcast sign(x) along rows
        # sign(c_k)*...
        X = sign_ck*X               # broadcast sign(ck) along columns
        # return sum along the last axis
        return np.sum(X, axis=-1)
        
    def get(self, index):
        """
        Returns the value stored at element :code:`index`, if no value
        is stored at that index, the generating function is used to
        calculate the value. If no generating function has been set,
        then a ValueError is raised.
        """
        try:
            return super().__getitem__(index)
        except IndexError:
            if self._generating_function is not None:
                return self._generating_function(index)
            else:
                raise ValueError("No generating function has been defined.")
        
    def fill(self, index):
        """
        Populates the coefficients out to, and including, :code:`index`.
        """
        for i in range(index):
            try:
                _ = self[index]
            except IndexError:
                self.append(self.get(i))
        
    def get_generating_function(self):
        return self._generating_function
    
    def set_generating_function(self, func):
        """
        Sets the generating function for the coefficients. If set, the
        generating function is called to calculate the coefficients. To
        bypass the generating function, see :code:`lookup`.
        
        Parameters
        ----------
        :param func: Function that calculates the power series expansion
            coefficient at a given index.
        :type func: Unary function, signature :code:`func(index)`.
        """
        self._generating_function = func
        
class LogisticSigmoidExpansion(PowerExpansion):
    @staticmethod
    def coeff_generating_function(n):
        if n == 0:
            rval = 0.5
        elif n % 2 == 1:
            rval = ((-1)**((n+3)/2) *
                    2 *
                    (4*np.pi)**(-((n+1)/2)) *
                    (4**((n+1)/2) - 1) *
                    zeta(n+1))
        else:
            rval = 0
        return rval
    
    @staticmethod
    def log_coeff_generating_function(n):
        if n == 0:
            rval = (1, np.log(0.5))
        elif n % 2 == 1:
            rval = ((-1)**((n+3)/2) *
                    2 *
                    (4*np.pi**2)**(-((n+1)/2)) *
                    (4**((n+1)/2) - 1) *
                    zeta(n+1))
            ln2 = np.log(2)
            pi_term = -(n+1)/2*(np.log(4) + 2*np.log(np.pi))
            if n > 14:
                pow4_term = (n+1)/2*np.log(4)
            else:
                pow4_term = np.log(4**((n+1)/2)-1)
            rval = ((-1)**((n+3)/2),
                    np.log(2) - (n+1)/2 * (np.log(4) + 2*np.log(np.pi))
        else:
            rval = 0
        return rval
    
    def __init__(self, *args, **kwds):
        super().__init__(*args, **kwds)
        self.set_generating_function(
            LogisticSigmoidExpansion.coeff_generating_function)

In [141]:
sig = LogisticSigmoidExpansion()

In [142]:
[sig.get(i) for i in range(10)]

[0.5,
 0.7853981633974484,
 0,
 -0.19327200601984604,
 0,
 0.06355835917038706,
 0,
 -0.020453014155255897,
 0,
 0.006529169858902421]

In [143]:
sig.fill(100000)

x = np.linspace(-5, 5, num=200)
plt.plot(x, sig(x));
plt.xlim(-5, 5);
plt.ylim(-2, 2);

OverflowError: (34, 'Result too large')

In [138]:
np.arange(5)[:, np.newaxis] + np.arange(5)

array([[0, 1, 2, 3, 4],
       [1, 2, 3, 4, 5],
       [2, 3, 4, 5, 6],
       [3, 4, 5, 6, 7],
       [4, 5, 6, 7, 8]])